In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as ss

In [ ]:
df = pd.read_pickle('data/processed/data.pkl')

In [ ]:
df

In [ ]:
# TODO: define smoothed envelope as feature

In [ ]:
# define training and validation data
X = []
Y = []
fs = 2500 # Hz
nsteps = 2500 # 0.xx s 
boxcar_width = 0.05 # s
boxcar_full = ss.lfilter(np.ones(int(fs * boxcar_width)), 1, df['ripple_true'])
for tstep in np.where(df['ripple_true'] == 1)[0]:
    inds = np.arange(- nsteps // 2, nsteps // 2) + tstep
    X += [np.c_[[df['LFP'][inds] - df['LFP'][inds].mean(), df['fLFP'][inds], df['eLFP'][inds]]].T]
    Y += [boxcar_full[inds + int(fs * boxcar_width / 2)]]
X = np.array(X)
Y = np.array(Y)
Y[Y >= 1] = 1.
Y = np.expand_dims(Y, axis=-1)

In [ ]:
X.shape, Y.shape

In [ ]:
fig, axes = plt.subplots(20, 1, figsize=(8, 32))
axes = axes.flatten()
for i, ax in enumerate(axes):
    ax.plot(X[i] / X[i].std(axis=0))
    ax.plot(Y[i])
    ax.axis(ax.axis('tight'))

In [ ]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.utils.vis_utils import plot_model
from keras.layers.normalization import BatchNormalization

In [ ]:
def generate_model_LSTM(input_shape):
    keras.backend.clear_session()

    # input layer
    inputs = keras.layers.Input(shape=input_shape)
    
    # conv layer
    x = keras.layers.Conv1D(20, kernel_size=10, strides=1, 
                           padding='same'
                           )(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('sigmoid')(x)
    x = keras.layers.Dropout(0.8)(x)
    
    # LSTM layer 1
    #x = keras.layers.LSTM(20, activation='relu', return_sequences=True)(x)
    x = keras.layers.LSTM(20, return_sequences=True)(x)
    #x = keras.layers.Activation('sigmoid')(x)
    x = keras.layers.BatchNormalization()(x)  
    x = keras.layers.Dropout(0.8)(x)
    
    # LSTM layer 2
    #x = keras.layers.GaussianNoise(1.5)(x)
    x = keras.layers.LSTM(20, return_sequences=True)(x)
    #x = keras.layers.Activation('sigmoid')(x)
    #x = keras.layers.Dropout(0.8)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.8)(x)
    
    # dense layer 1
    # x = keras.layers.TimeDistributed(Dense(20, activation='sigmoid'))(x)
    
    # dense output layer
    predictions = keras.layers.TimeDistributed(
        Dense(1, activation = "sigmoid"))(x) # time distributed  (sigmoid)
    
    # Define model
    model = keras.models.Model(inputs=inputs, outputs=predictions)

    #opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
    opt = keras.optimizers.Adam(lr=0.005)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])
           #model.compile(loss=['categorical_crossentropy'],
    #              optimizer=keras.optimizers.Adam(lr=0.01),
    #              metrics=['accuracy'],
    #              # sample_weight_mode='temporal',
    #              )

    return model

In [ ]:
model = generate_model_LSTM(input_shape=X[0].shape)

In [ ]:
model.summary()

In [ ]:
X.shape, Y.shape

In [ ]:
history = model.fit(X, Y, batch_size=16, epochs=10, validation_split=0.1)

In [ ]:
plt.figure()
plt.plot(history.history['loss'], '-o', label='loss')
plt.plot(history.history['accuracy'], '-o', label='accuracy')
plt.plot(history.history['val_accuracy'], '-o', label='val_accuracy')
plt.legend()

In [ ]:
# run prediction on a sample
Ypred = model.predict(X[-20:])

In [ ]:
# compare prediction to ground truth
fig, axes = plt.subplots(20, 1, figsize=(8, 16))
for i in range(axes.size):
    axes[i].plot(Y[-20:][i].flatten())
    axes[i].plot(Ypred[i].flatten())    